In [ ]:
import math
from torch import nn
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics import Accuracy, ConfusionMatrix, F1Score, Recall, Precision
import torchinfo
import os
from settings import *
from Data.MowingDetectionDataset_Parcelized import *
from System.pos_enc import *
from System.Engine import *
from helpers.various import *

In [ ]:
transforms = A.Compose([
        #A.Normalize(mean=[-0.3357, -0.3259],#####how many channels to include?
                   # std=[0.4688, 0.4685]),
        ToTensorV2()
    ])

In [ ]:
train_dt = ParcelMowingDataset(root_path = root_dataset_path,labels_path = path_train, trsm = transforms,max_lenght_to_pad=50,
                nan_free = True,concat_mask = False,mode = 'pad')
train_dloader = DataLoader(train_dt, sampler=ImbalancedDatasetSampler(train_dt), 
                           batch_size=train_batch, collate_fn = collate_fn, 
                           shuffle=False, num_workers=0)

In [ ]:
val_dt = ParcelMowingDataset(root_dataset_path,labels_path = path_val, trsm = transforms, max_lenght_to_pad=50,
                nan_free = True,concat_mask = False,mode = 'pad')
val_dloader = DataLoader(val_dt, batch_size=val_batch, shuffle=False, num_workers=0)

In [ ]:
test_dt = ParcelMowingDataset(root_dataset_path,labels_path = path_test, trsm = transforms, max_lenght_to_pad=50,
                nan_free = True,concat_mask = False,mode = 'pad')
test_dloader = DataLoader(test_dt, batch_size=test_batch, shuffle=False, num_workers=0)

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.tuner.tuning import Tuner


callbacks = [
    EarlyStopping(monitor="accuracy_m/val", mode="max", patience=100),
    ModelCheckpoint(monitor="accuracy_m/val", mode="max", save_last=True),
    LearningRateMonitor(logging_interval='epoch'),
]

model = Model()
trainer = pl.Trainer(
    accelerator="gpu", 
    devices=1,
    max_epochs=number_of_epochs,
    callbacks=callbacks,
    default_root_dir=save_dir,
    enable_progress_bar = False,
    #detect_anomaly=True,
)

trainer.fit(model, train_dataloaders=train_dloader, val_dataloaders=val_dloader)

In [ ]:
best_model = Model.load_from_checkpoint('./epoch=22-step=2921.ckpt')
trainer = pl.Trainer(accelerator="gpu", 
    devices=1)
trainer.test(best_model, dataloaders=test_dloader)

In [ ]:
#####to do!!!!####
from sklearn.metrics import ConfusionMatrixDisplay
from matplotlib import pyplot as plt


class_labels = ['0 times', '1 times','2 times', '3 times', '4 times']
cm = best_model.test_confusion_matrix.compute().cpu().numpy()

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=class_labels )
plt.figure(figsize=(5,5), dpi=100)
ax = plt.axes()

disp.plot(ax=ax)
plt.xticks(rotation=90)
plt.show()